In [1]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import math

In [2]:
# parameters
starting_tank = 10000
max_tank = 25000
roof_capture_per_inch_rain = 1860      #    3000 sq ft * 0.62 assuming 12" depth  https://www.surfrider.org/coastal-blog/entry/calculate-rainwater-harvesting-potential-area-needed-to-absorb-it 
exp_lambda = 0.4261                    # lambda for exponential distribution that fits monthly rainfall

In [4]:
min_levels = []

water_df = pd.DataFrame(columns = ['month', 'ending_water_level', 'month_rainfall', 'rain_harvested', 'water_used', 'rainfall_capture_eff_pct', 'random_num'])

sim_months = 360

for i in range(1000):             # 1000 simulations of 360 months
    water_df = pd.DataFrame(columns = ['month', 'ending_water_level', 'month_rainfall', 'rain_harvested', 'water_used', 'rainfall_capture_eff_pct', 'random_num'])

    for j in range(0, sim_months):                                    # run a simulation for 30 years
        
        ran_num = random.random()                                     # random number between 0 and 1
        month_rainfall = -(1/exp_lambda) * (math.log(ran_num))                 # exponential distribution, lambda is 0.4261
        
        rainfall_capture_eff_pct = 0.9 + ((8*(1-ran_num))/100)             # capture efficiency varies from 90-98% based on severity of downfall
        rain_harvested = month_rainfall * roof_capture_per_inch_rain * rainfall_capture_eff_pct # how much rain * how much water captured on roof * efficiency of capture between 90-98%

        water_used = random.uniform(4000,5200)                        # monthly water usage is random between 4000 and 5200

        if j == 0:                                                    # first month water level at 10000
            starting_water_level = starting_tank
        if j > 0:
            starting_water_level = water_df.iloc[- 1, 1]                 # future months water level begins at last month's level

        ending_water_level = starting_water_level + rain_harvested - water_used    

        if ending_water_level < 0:                                    # water level can't go below 0
            ending_water_level = 0
        if ending_water_level > 25000:                                # water level can't go above 25,000. No penalty for hitting 25,000
            ending_water_level = 25000
        water_df.loc[i] = [ j + 1, ending_water_level, month_rainfall, rain_harvested, water_used, rainfall_capture_eff_pct, ran_num]
    min_levels.append(water_df['ending_water_level'].min())


In [5]:
print(min(min_levels))
print(sum(min_levels)/len(min_levels))
print(max(min_levels))

0.0
8747.218198240016
25000.0
